In [2]:
import os
import joblib
import numpy as np
import pandas as pd
from typing import List, Optional
from pandas.api.types import is_numeric_dtype, is_datetime64_any_dtype

# --------------- Config ---------------
DATE = "20251107"
PRED_DATA_PATH = f"data/inference/{DATE}/inference_data.csv"  # your input dataframe (one row per game)
MODEL_DIR      = "./models/cbb_models_fast"           # where model_{target}.joblib live
OUT_CSV        = f"data/predictions/{DATE}/preds.csv"            # optional: will be created
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)

TARGETS = [
    "home_1h", "away_1h", "home_2h", "away_2h",
    "home_score", "away_score", "home_margin", "away_margin"
]

TARGETS = [
    "home_1h", "away_1h", "home_2h", "away_2h",
    "home_score", "away_score", "home_margin", "away_margin"
]

PASSTHRU_COLS = ["game_id","game_id_y","date","date_utc","time_utc","home","away"]
NONNEG_TARGETS = {"home_1h","away_1h","home_2h","away_2h","home_score","away_score"}

def choose_date_column(df: pd.DataFrame) -> Optional[str]:
    if "date" in df.columns: return "date"
    if "date_utc" in df.columns: return "date_utc"
    return None

def _align_X_for_model(raw: pd.DataFrame, mdl) -> pd.DataFrame:
    """
    Build model input with the exact features LightGBM expects:
      • read expected names from Booster
      • create missing as NaN
      • convert any datetime columns to epoch seconds (float)
      • cast to float32
    """
    # Always use Booster names if available
    feat_list = None
    if hasattr(mdl, "_Booster") and mdl._Booster is not None:
        try:
            feat_list = list(mdl._Booster.feature_name())
        except Exception:
            pass
    if feat_list is None:
        feat_list = getattr(mdl, "feature_name_", None)

    if not feat_list:  # rare fallback
        cols = [c for c in raw.columns if is_numeric_dtype(raw[c])]
        return raw[cols].astype(np.float32)

    # Ensure all expected features exist
    for c in feat_list:
        if c not in raw.columns:
            raw[c] = np.nan

    # Convert datetime features to epoch seconds
    for c in feat_list:
        s = raw[c]
        if is_datetime64_any_dtype(s):
            s_dt = pd.to_datetime(s, errors="coerce")
            # convert to ns → float seconds; keep NaT as NaN
            vals = s_dt.view("int64").astype("float64")
            vals[s_dt.isna().values] = np.nan
            raw[c] = vals / 1e9  # seconds; matches “numeric” expectation

    # Keep exactly expected features, ordered, as float32
    X = raw[feat_list].astype(np.float32)
    return X

def run_inference():
    raw = pd.read_csv(PRED_DATA_PATH)

    # parse dates for display only
    for c in ("date","date_utc"):
        if c in raw.columns:
            raw[c] = pd.to_datetime(raw[c], errors="coerce")

    if "season" not in raw.columns:
        # use the season your data belongs to; constant is fine for future games
        raw["season"] = 2026
    # passthrough columns
    keep_cols = [c for c in PASSTHRU_COLS if c in raw.columns]
    id_col = "game_id" if "game_id" in keep_cols else ("game_id_y" if "game_id_y" in keep_cols else None)
    date_col = choose_date_column(raw)

    base_out_cols = []
    if id_col: base_out_cols.append(id_col)
    if date_col: base_out_cols.append(date_col)
    if "home" in raw.columns: base_out_cols.append("home")
    if "away" in raw.columns: base_out_cols.append("away")
    out = raw[base_out_cols].copy()
    out['season'] = 2026

    # predictions per target
    for tgt in TARGETS:
        path = os.path.join(MODEL_DIR, f"model_{tgt}.joblib")
        if not os.path.exists(path):
            out[f"pred_{tgt}"] = np.nan
            continue

        mdl = joblib.load(path)
        X = _align_X_for_model(raw, mdl)  # <-- guarantees numeric float32 only
        preds = mdl.predict(X)
        if tgt in NONNEG_TARGETS:
            preds = np.maximum(preds, 0.0)
        out[f"pred_{tgt}"] = preds.astype(np.float32)

    # tidy columns
    if "game_id_y" in out.columns and "game_id" not in out.columns:
        out = out.rename(columns={"game_id_y": "game_id"})

    pred_cols = [c for c in out.columns if c.startswith("pred_")]
    out = out[ [c for c in ["game_id", date_col, "home", "away"] if c in out.columns] + sorted(pred_cols) ]

    os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
    out['date'] = DATE
    out.to_csv(OUT_CSV, index=False)
    print(f"Saved predictions → {OUT_CSV}")
    return out

In [3]:
run_inference()

Saved predictions → data/predictions/20251107/preds.csv


,game_id,date,home,away,pred_away_1h,pred_away_2h,pred_away_margin,pred_away_score,pred_home_1h,pred_home_2h,pred_home_margin,pred_home_score
0,401809073.0,20251107,Mount St. Mary,Bucknell,32.124393,33.631420,1.599726,74.733635,33.712967,36.627571,-1.599726,64.256294
1,NaN,20251107,UCLA,Pepperdine,35.810677,38.626015,-9.203074,74.374298,38.410187,39.393326,9.203074,82.523979
2,401808948.0,20251107,Louisiana,Southeastern Louisiana,27.397888,29.884651,-10.308118,61.463764,33.980698,38.515141,10.308118,77.357971
3,401813272.0,20251107,Maryland,Georgetown,29.622671,35.106682,-3.645890,66.905914,35.860783,38.894547,3.645890,74.843193
4,NaN,20251107,DePaul,Stonehill,27.820349,29.793819,-16.226776,57.971306,40.545528,43.417950,16.226776,84.119011
5,401826746.0,20251107,Eastern Illinois,Nicholls St.,27.314840,30.511293,-8.631918,59.521114,37.811474,40.509914,8.631918,72.219490
6,NaN,20251107,Rhode Island,Tulsa,32.978172,33.762154,-13.169809,68.885979,36.713966,45.252686,13.169809,81.731468
7,401824356.0,20251107,Northwestern,Boston University,28.252789,35.937164,-10.478326,61.264206,37.068184,38.949795,10.478326,72.125702
8,NaN,20251107,Navy,Yale,28.185595,33.595516,-17.875828,64.472878,38.221905,41.546379,17.875828,75.515617
9,401819882.0,20251107,Buffalo,Green Bay,26.204460,29.654749,-19.623049,59.703159,32.501362,40.285610,19.623049,73.543259
